- TF: 문서당 단어가 몇 개 있는지
    - 0문서: 사과
    - 1문서: 바나나
    - 2문서: 바나나 바나나 과일
    - ___과일 바나나 사과
    - 0___0___0____1
    - 1___0___1____0
    - 2___0___1____1
- DF: 문서 당 단어 존재
    - 과일___1
    - 바나나___3
    - 사과___1
    - 어느 문서에 하나 나온 단어일수록 값 큼
    - 모든 문서에 나온 단어가 값이 낮음(중요도 낮음)
    - 특정 문서만에 많은 단어: 가중치 높음
- IDF: DF의 인버스. 로그, 분모에 1....취한 형태
- TF-TDF: 위 2개 행렬곱

In [1]:
# 참고 : https://wikidocs.net/31698

import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

In [2]:
docs = [
  '사과',
  '바나나',
  '사과 바나나 쥬스',
  '바나나 바나나 과일',
  '수박'
] 

In [3]:
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
N = len(docs) # 총 문서의 수

In [4]:
def tf(t, d):
    return d.count(t)

In [5]:
def df(t):
    df = 0
    for doc in docs:
        df = df + (t in doc)
    return df

In [6]:
#################################### tf
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(df(t))

In [7]:
df_ = pd.DataFrame(result, index = vocab, columns = ["DF"])
df_

,DF
과일,1
바나나,3
사과,2
수박,1
쥬스,1


In [8]:
def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

In [9]:
def tfidf(t, d):
    return tf(t,d)* idf(t)

In [10]:
#################################### tf
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]        
        result[-1].append(tf(t, d))
tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일,바나나,사과,수박,쥬스
0,0,0,1,0,0
1,0,1,0,0,0
2,0,1,1,0,1
3,1,2,0,0,0
4,0,0,0,1,0


In [11]:
#################################### df
###############
idf_result= pd.DataFrame(result, columns = vocab)
idf_result.astype(float) 
for i in range(N):
    idf_result.iloc[i]= \
        idf_result.iloc[i].map(lambda x: round(log(N/(1+x)),2))
    #idf_result.iloc[i]=log(4/(idf_result.iloc[i]+1.0))

In [12]:
#################################### idf
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

In [13]:
idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
과일,0.916291
바나나,0.223144
사과,0.510826
수박,0.916291
쥬스,0.916291


In [14]:
#################################### tf-idf
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))

In [15]:
tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일,바나나,사과,수박,쥬스
0,0.000000,0.000000,0.510826,0.000000,0.000000
1,0.000000,0.223144,0.000000,0.000000,0.000000
2,0.000000,0.223144,0.510826,0.000000,0.916291
3,0.916291,0.446287,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.916291,0.000000
